In [2]:
from run_gwas import read_loader_in, order_cols, run_plink1
import numpy as np
import pandas as pd
from pandas_plink import read_plink1_bin, write_plink1_bin
import os
import ast
from os.path import isfile, join
import re
import string
import random
from LabQueue.qp import qp
from LabData.DataLoaders.CGMLoader import CGMLoader
from GeneticsPipeline.helpers_genetic import read_status_table, run_plink2, required_memory_gb
from LabUtils.Scripts.shell_commands_execute import ShellCommandsExecute
from LabUtils.addloglevels import sethandlers
from GeneticsPipeline.config import plink19_bin, plink2_bin, qp_running_dir
from pandas_plink import write_plink1_bin
import matplotlib.pyplot as plt

In [3]:
##Only run once
sethandlers()

(None, None)

In [10]:
##EA1c with outliers removed
batchedphenos = pd.read_csv("/net/mraid20/export/jasmine/zach/height_gwas/all_gwas/phenos_batched/batch0.txt", sep = "\t")
justeA1C = batchedphenos.loc[:,["IID", "ea1c"]]
justeA1C.to_csv(batched_pheno_dir + "ea1c" + ".txt", sep="\t", index=False, header=True, na_rep = "NA")

In [7]:
##Raw Ea1c with outliers in place
batched_pheno_dir = "/net/mraid20/export/jasmine/zach/height_gwas/all_gwas/phenos_batched/"
status_table = read_status_table()
status_table = status_table[status_table.passed_qc].copy()
df = pd.read_csv("/net/mraid20/export/genie/LabData/Analyses/ayyak/CGM/iglu/iglu_no_tails.csv").set_index("id").drop("Unnamed: 0", axis=1)
##throw away second part of index
df.index = list(map(lambda longName: longName.split("/")[0], df.index.values))
df.index.name = "RegistrationCode"
#df = df.loc[~df.index.duplicated(keep = "first"),:] #also try mean
df = df.groupby("RegistrationCode").mean()
df = df.loc[:, ["ea1c"]]
temp_index = df.reset_index().RegistrationCode.apply(status_table.set_index('RegistrationCode').gencove_id.to_dict().get)
temp_index.name = "IID"
df = df.set_index(temp_index)
df = order_cols(df, keep_fid = False)
df.to_csv(batched_pheno_dir + "ea1c_raw" + ".txt", sep="\t", index=False, header=True, na_rep = "NA")

In [8]:
bfile_loc = "/net/mraid20/export/genie/10K/genetics/Gencove/allsamples"
batched_pheno_dir = "/net/mraid20/export/jasmine/zach/height_gwas/all_gwas/phenos_batched/"
outfile_loc = "/net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_results/eac_noqc_with_outliers"
cmd = plink2_bin +' --bfile '+ bfile_loc + ' --pheno /net/mraid20/export/jasmine/zach/height_gwas/all_gwas/phenos_batched/ea1c_raw' + '.txt' + ' --linear hide-covar --out ' + outfile_loc

In [9]:
##make sure you only ask for memory based on the qc'd genetics data
run_plink2(cmd, "gwas", required_memory_gb("/net/mraid20/export/genie/10K/genetics/Gencove/allsamples_qc.bed"), threads = 32)

2022-17-10 14:56:48 INFO qp.py:1118 Sending to queue (mem 41G, threads 32): (gwas.._C8k) /usr/wisdom/python-3.7.4/bin/python3.7 -W ignore /home/zacharyl/Desktop/LabQueue/LabQueue/qworker.py /net/mraid20/export/genie/10K/genetics/Logs/gwas.._c4e56c46c66a46e2a0808dd644490fc1/0.dat;
2022-17-10 14:56:58 INFO qp.py:1034 Job: gwas.._C8k running on genie46
2022-17-10 15:15:27 INFO CommandStatus.py:96 
	/usr/wisdom/python-3.7.4/bin/python3.7 -W ignore /home/zacharyl/Desktop/LabQueue/LabQueue/qworker.py /net/mraid20/export/genie/10K/genetics/Logs/gwas.._c4e56c46c66a46e2a0808dd644490fc1/0.dat;
	Running gwas


In [10]:
res = pd.read_csv("/net/mraid20/export/jasmine/zach/height_gwas/all_gwas/gwas_results/eac_noqc_with_outliers.ea1c.glm.linear", sep = "\t")
res_sig = res.loc[res.P < (5*10**(-8)),:]
len(res_sig)

9472

In [11]:
their_res = pd.read_csv("~/Desktop/pheno_cgm_assoc.csv")
their_res_eac = their_res.loc[their_res.phenotype == "eA1C",:]
their_res_eac_sig = their_res_eac[their_res_eac.P < (5*10**(-8))]

In [12]:
merged_all = pd.merge(left = res, left_on = "ID", right = their_res_eac, right_on = "SNP", how = "inner")

merged_sig = pd.merge(left = res_sig, left_on = "ID", right = their_res_eac_sig, right_on = "SNP", how = "inner")

In [16]:
#len(merged_sig)
#len(merged_all)
len(merged_sig)/len(their_res_eac_sig)
#len(merged_all)/len(their_res_eac)

0.34615384615384615

In [18]:
merged.columns

Index(['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'A1', 'TEST', 'OBS_CT', 'BETA',
       'SE', 'T_STAT', 'P_x', 'SNP', 'P_y', 'phenotype', 'chrom', 'pos'],
      dtype='object')

In [19]:
merged.head()

,#CHROM,POS,ID,REF,ALT,A1,TEST,OBS_CT,BETA,SE,T_STAT,P_x,SNP,P_y,phenotype,chrom,pos
0,1,2347465,rs77529941,T,C,C,ADD,7327,-0.073967,0.025988,-2.84618,0.004437,rs77529941,0.000085,eA1C,1,2347465
1,1,4594998,rs34231005,T,G,G,ADD,7327,-0.016053,0.007601,-2.11180,0.034738,rs34231005,0.000090,eA1C,1,4594998
2,1,13835765,rs11576472,C,T,T,ADD,7327,-0.037098,0.014039,-2.64240,0.008250,rs11576472,0.000070,eA1C,1,13835765
3,1,14630083,rs75045455,C,T,T,ADD,7327,-0.040610,0.022009,-1.84514,0.065057,rs75045455,0.000056,eA1C,1,14630083
4,1,14707476,rs820666,G,A,A,ADD,7327,0.035940,0.016378,2.19437,0.028241,rs820666,0.000078,eA1C,1,14707476
